### Importando bibliotecas necessárias
- `pandas`: Manipulação e análise de dados.
- `numpy`: Operações numéricas e matrizes.
- `matplotlib.pyplot`: Geração de gráficos básicos.
- `seaborn`: Visualizações estatísticas avançadas.


In [344]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Carregando a base de dados

In [ ]:
# Carregando dataset com informações sobre livros
dfBooks = pd.read_csv('archive/Books.csv')

C:\Users\Gabriel Lopes\AppData\Local\Temp\ipykernel_16440\2782996754.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  dfBooks = pd.read_csv('archive/Books.csv')


In [ ]:
# Carregando dataset com informações sobre usuários
dfUsers = pd.read_csv('archive/Users.csv')

In [ ]:
# Carregando dataset com informações sobre as avaliações dos livros
dfRatings = pd.read_csv('archive/Ratings.csv')

### Primeiras impressões dos dados
Abaixo, visualizamos as primeiras linhas de cada conjunto de dados:
- `dfUsers`: Contém informações sobre os usuários.
- `dfBooks`: Contém detalhes sobre os livros, como título, autor, ano de publicação e editora.
- `dfRatings`: Registra as avaliações dadas pelos usuários aos livros.


In [348]:
dfUsers.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [349]:
dfBooks.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [350]:
dfRatings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


### Renomeando colunas
Os nomes das colunas foram padronizados para facilitar a manipulação dos dados:
- `User-ID` → `user_id`
- `Book-Author` → `author`
- `Year-Of-Publication` → `year`
- `Publisher` → `publisher`
- `Book-Title` → `title`


In [ ]:
# Renomeando colunas para padronizar os nomes e facilitar a análise
dfBooks.rename(columns={'User-ID': 'user_id', 'Book-Author': 'author', 
                        'Year-Of-Publication': 'year', 'Publisher': 'publisher', 
                        'Book-Title': 'title'}, inplace=True)

dfRatings.rename(columns={'User-ID': 'user_id', 'Book-Rating': 'rating'}, inplace=True)

### Verificando a frequência de avaliações dos usuários
Aqui, contamos quantas avaliações cada usuário fez para identificar padrões de engajamento.


In [353]:
count = dfRatings['user_id'].value_counts()

In [354]:
count

user_id
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: count, Length: 105283, dtype: int64

Podemos perceber que há usuários que avaliam poucos livros e outros que mantêm um hábito de leitura mais intenso, avaliando muitos livros.  
Para nossa análise, focaremos nos usuários mais ativos, pois suas avaliações podem servir como parâmetro para o modelo.

### Filtrando usuários com mais de 250 avaliações
Como queremos dados de usuários mais engajados, filtramos aqueles que avaliaram mais de 250 livros.

In [355]:
dfRatings = dfRatings[dfRatings['user_id'].isin(count[count > 250].index)]

In [356]:
dfRatings.shape

(480219, 3)

Após a filtragem, removemos aproximadamente metade dos dados, deixando apenas usuários com mais de 250 avaliações.

### Unindo as avaliações com os dados dos livros
Agora, juntamos as avaliações filtradas com as informações dos livros, usando a chave `ISBN`.

In [357]:
rating_books = dfRatings.merge(dfBooks, on='ISBN')

In [358]:
rating_books.shape

(444950, 10)

In [359]:
num_rating = rating_books.groupby('title')['rating'].count().reset_index()

### Contagem de avaliações por livro
Aqui, calculamos quantas avaliações cada livro recebeu no dataset.

In [360]:
num_rating.rename(columns={'rating': 'num_rating'}, inplace=True)

In [361]:
num_rating.shape

(152863, 2)

### Unindo as a contagem de avaliações com os dados dos livros
Agora, juntamos as contagens de avaliações filtradas com as informações dos livros, usando a chave `title`.

In [362]:
final_data = rating_books.merge(num_rating, on='title')

In [363]:
final_data.shape

(444950, 11)

### Filtrando livros com 60 avaliações ou mais
Como queremos dados de livros com hatings maior, filtramos aqueles que somam 60 avaliações ou mais.

In [364]:
final_data = final_data[final_data['num_rating'] >= 60]

In [365]:
# antes de apagar dados duplicados
final_data.shape

(37420, 11)

In [366]:
final_data = final_data.drop_duplicates(['user_id', 'title'])

In [367]:
# depois de apagar os dados duplicados
final_data.shape

(35969, 11)

### Primeiras impressoes depois do dataset tratado

In [368]:
final_data.head(2)

,user_id,ISBN,rating,title,author,year,publisher,Image-URL-S,Image-URL-M,Image-URL-L,num_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,73
13,277427,0060930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,http://images.amazon.com/images/P/0060930535.0...,http://images.amazon.com/images/P/0060930535.0...,117


### Renomeando e excluindo algumas colunas redundates

In [369]:
final_data.drop(columns=['Image-URL-S', 'Image-URL-M'], inplace=True)

In [370]:
final_data.shape

(35969, 9)

In [371]:
final_data.rename(columns={'Image-URL-L': 'image_url'}, inplace=True)

### "Pivotando" a tabela

In [372]:
book_pivot = final_data.pivot_table(columns='user_id', index='title', values='rating').fillna(0)

In [373]:
book_pivot

user_id,254,2276,2766,3363,3757,4385,6251,6543,6575,7158,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wish You Well,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Without Remorse,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Wuthering Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


A função pivot_table do pandas é frequentemente usada em sistemas de recomendação para transformar dados de formato longo (onde cada linha representa uma interação entre um usuário e um item) em uma matriz de utilidade (ou matriz de interação), onde as linhas representam usuários, as colunas representam itens, e os valores representam as interações (como classificações, visualizações, etc.).

Essa matriz de utilidade é essencial para muitos algoritmos de recomendação, especialmente aqueles baseados em filtragem colaborativa, como o K-Nearest Neighbors (KNN) e a decomposição em valores singulares (SVD).

In [374]:
from scipy.sparse import csr_matrix

A função csr_matrix do módulo scipy.sparse é usada para criar uma matriz esparsa no formato CSR (Compressed Sparse Row). Esse formato é eficiente em termos de memória e operações de matriz, especialmente quando a matriz contém muitos zeros, o que é comum em sistemas de recomendação.

Vantagens do Formato CSR

- Eficiência de Memória: Armazena apenas os elementos não-zero, economizando memória.

- Operações Rápidas: Permite operações eficientes de fatiamento e aritmética de matriz.

- Conversão Fácil: Pode ser facilmente convertida para outros formatos esparsos.

In [375]:
book_pivot_normalized = book_pivot.sub(book_pivot.mean(axis=1), axis=0).fillna(0)
book_pivot_sparse = csr_matrix(book_pivot_normalized)

In [376]:
from sklearn.neighbors import NearestNeighbors

**Como Funciona o Algoritmo NearestNeighbors** <br>

O algoritmo de NearestNeighbors (vizinhos mais próximos) é uma técnica de aprendizado de máquina usada para encontrar os pontos de dados mais próximos em um espaço de características. Ele é amplamente utilizado em sistemas de recomendação, reconhecimento de padrões, e outras aplicações que envolvem a busca de similaridade.

O algoritmo NearestNeighbors pode ser configurado para usar diferentes métricas de distância e algoritmos de busca para encontrar os vizinhos mais próximos.

Métricas de Distância:

- euclidean: Distância Euclidiana.
- manhattan: Distância de Manhattan (ou L1).
- cosine: Distância do Cosseno.
- Outras métricas personalizadas.

Algoritmos de Busca:

- brute: Busca exaustiva (força bruta).
- kd_tree: Árvore KD (K-Dimensional).
- ball_tree: Árvore de Bolas.
- auto: Seleciona automaticamente o melhor algoritmo com base nos dados.
- Número de Vizinhos:

n_neighbors: Número de vizinhos mais próximos a serem encontrados.

In [377]:
model = NearestNeighbors(algorithm='auto', metric='cosine')

In [378]:
# Crie uma cópia para teste
train_data = book_pivot_sparse.copy()
test_data = book_pivot.copy()  # Mantém como DataFrame para facilitar acesso

# Oculte 20% das avaliações aleatoriamente
mask = np.random.rand(*book_pivot.shape) < 0.2  # 20% de teste
train_data[mask] = 0  # Zera as avaliações ocultas no treino
train_data = csr_matrix(train_data)  # Converte novamente para esparso

In [379]:
model.fit(train_data)

NearestNeighbors(metric='cosine')

In [380]:
distance, suggestions = model.kneighbors(book_pivot.iloc[1,:].values.reshape(1, -1), n_neighbors=6)

Detalhes de `book_pivot.iloc[237, :]`:

- Contexto: `book_pivot` é um **DataFrame**, uma matriz de utilidade onde:  
  - Linhas representam livros.  
  - Colunas representam usuários.  

- Seleção: `iloc[237, :]` seleciona a **linha 237** do DataFrame `book_pivot`. Essa linha contém as características do livro com índice **237**.

Transformação com `.values.reshape(1, -1)`:

- `.values`: Converte a linha selecionada em um **array NumPy**.  
- `.reshape(1, -1)`: Reorganiza o array em uma **matriz 2D** com:  
  - Uma única linha.  
  - Colunas suficientes para acomodar todos os elementos (-1 indica que o número de colunas é calculado automaticamente).  

  Essa transformação é necessária porque a função `kneighbors` requer que a entrada seja uma **matriz 2D**.

Parâmetro `n_neighbors=6`:

- Indica que a busca será feita para encontrar os **6 vizinhos mais próximos** do ponto de consulta.


In [381]:
distance

array([[0.14494755, 0.67470549, 0.71850371, 0.74760223, 0.74874527,
        0.7638981 ]])

### Lista de indicies dos livros recomendados

In [382]:
suggestions

array([[  1,  22,  67, 171, 220, 357]], dtype=int64)

**Medindo o desempenho do modelo**

Entrada: O código pega as características do livro na linha 237 do DataFrame book_pivot.

Processamento: Usa o modelo NearestNeighbors para encontrar os 6 livros mais próximos (vizinhos) ao livro na linha 237, com base nas características fornecidas.

Saída: A função kneighbors retorna duas coisas:

distance: Um array contendo as distâncias dos 6 livros mais próximos ao livro de consulta.

suggestions: Um array contendo os índices dos 6 livros mais próximos no DataFrame book_pivot.

In [383]:
for i in range(len(suggestions)):
    print(book_pivot.index[suggestions[i]])

Index(['1st to Die: A Novel', 'Along Came a Spider (Alex Cross Novels)',
       'Cradle and All', 'Lightning', 'Remember Me', 'The Summons'],
      dtype='object', name='title')


In [384]:
books_name = book_pivot.index

In [385]:
books_name

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Is for Alibi (Kinsey Millhone Mysteries (Paperback))',
       'A Map of the World', 'A Painted House',
       ...
       'Whispers', 'White Oleander : A Novel',
       'White Oleander : A Novel (Oprah's Book Club)',
       'Wicked: The Life and Times of the Wicked Witch of the West',
       'Wild Animus', 'Wish You Well', 'Without Remorse', 'Wuthering Heights',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       '\O\" Is for Outlaw"'],
      dtype='object', name='title', length=411)


Exemplo prático:

Suponha que:

- `suggestions` seja `[237, 42, 15, 89, 300]`, ou seja, uma lista com 5 índices de livros que são "sugeridos" como similares (talvez encontrados pelo `kneighbors`).  
- `book_pivot.index` seja uma lista como `["O Senhor dos Anéis", "Harry Potter", "Código Da Vinci", ...]`, onde cada posição corresponde a um livro.

O loop fará o seguinte:

- Primeira iteração (`i = 0`):  
  `print(book_pivot.index[suggestions[0]])` imprime o livro na posição `suggestions[0]` (237). Se `book_pivot.index[237]` for "O Senhor dos Anéis", a saída será:  
  `O Senhor dos Anéis`

- Segunda iteração (`i = 1`):  
  `print(book_pivot.index[suggestions[1]])` imprime o livro na posição `suggestions[1]` (42). Se `book_pivot.index[42]` for "Harry Potter", a saída será:  
  `Harry Potter`

E assim por diante, até que todos os índices em `suggestions` sejam processados. No final, o código simplesmente lista os títulos ou identificadores dos livros que foram considerados "sugestões" ou "vizinhos mais próximos".


In [386]:
import pickle

### Exportando os dados que são necessarios para fazer o webapp com streamlit

In [387]:
pickle.dump(books_name, open('artefatos/book_name.pkl', 'wb'))
pickle.dump(model, open('artefatos/model.pkl', 'wb'))
pickle.dump(final_data, open('artefatos/final_data.pkl', 'wb'))
pickle.dump(book_pivot, open('artefatos/book_pivot.pkl', 'wb'))

### Funções para avaliar o desempenho do modelo

In [388]:
def evaluate_recommendations(book_title, model, train_data, test_data, k=6):
    # Encontre o ID do livro
    book_id = np.where(book_pivot.index == book_title)[0][0]
    
    # Obtenha recomendações
    distances, suggestions = model.kneighbors(train_data[book_id], n_neighbors=k)
    recommended_books = book_pivot.index[suggestions[0]]
    print(f"Recomendações para '{book_title}':", recommended_books.tolist())
    
    # Encontre livros relevantes (avaliações altas no conjunto completo)
    user_ratings = test_data.iloc[book_id, :].values  # Avaliações reais do livro
    true_likes = test_data.index[test_data.loc[:, test_data.columns[user_ratings > 7]].notna().any(axis=1)].tolist()
    true_likes = [book for book in true_likes if book != book_title]  # Exclui o próprio livro
    
    # Calcule acertos
    hits = len(set(recommended_books) & set(true_likes))
    precision = hits / k if k > 0 else 0
    recall = hits / len(true_likes) if true_likes else 0
    
    return precision, recall, true_likes

# Teste para um livro
book_title = "1st to Die: A Novel"
precision, recall, true_likes = evaluate_recommendations(book_title, model, train_data, test_data)
print(f"Precisão@{6}: {precision}, Recall@{6}: {recall}")
print(f"Livros relevantes reais: {true_likes}")

Recomendações para '1st to Die: A Novel': ['1st to Die: A Novel', 'Along Came a Spider (Alex Cross Novels)', 'Lightning', 'While My Pretty One Sleeps', 'Remember Me', 'Night Whispers']
Precisão@6: 0.8333333333333334, Recall@6: 0.012195121951219513
Livros relevantes reais: ['1984', '2nd Chance', '4 Blondes', 'A Bend in the Road', 'A Case of Need', 'A Child Called \\It\\": One Child\'s Courage to Survive"', 'A Is for Alibi (Kinsey Millhone Mysteries (Paperback))', 'A Map of the World', 'A Painted House', 'A Prayer for Owen Meany', "A Thousand Acres (Ballantine Reader's Circle)", 'A Time to Kill', 'A Walk to Remember', 'A Widow for One Year', 'A Wrinkle in Time', "ANGELA'S ASHES", 'Absolute Power', 'Airframe', 'All Around the Town', 'All I Really Need to Know', 'All That Remains (Kay Scarpetta Mysteries (Paperback))', 'Along Came a Spider (Alex Cross Novels)', 'American Gods', "Angela's Ashes (MMP) : A Memoir", "Angela's Ashes: A Memoir", 'Angels', 'Angels &amp; Demons', 'Anne Frank: The 

In [389]:
books_to_test = ["1st to Die: A Novel", "The Bridges of Madison County", "The Brethren"]
precisions, recalls = [], []
for book in books_to_test:
    p, r, _ = evaluate_recommendations(book, model, train_data, book_pivot)
    precisions.append(p)
    recalls.append(r)
print(f"Média Precisão@6: {np.mean(precisions)}, Média Recall@6: {np.mean(recalls)}")

Recomendações para '1st to Die: A Novel': ['1st to Die: A Novel', 'Along Came a Spider (Alex Cross Novels)', 'Lightning', 'While My Pretty One Sleeps', 'Remember Me', 'Night Whispers']
Recomendações para 'The Bridges of Madison County': ['The Bridges of Madison County', 'Mirror Image', '4 Blondes', 'Fall On Your Knees (Oprah #45)', 'The King of Torts', "Vinegar Hill (Oprah's Book Club (Paperback))"]
Recomendações para 'The Brethren': ['The Brethren', 'The Partner', 'Presumed Innocent', 'The Pelican Brief', 'The Testament', 'Red Storm Rising']
Média Precisão@6: 0.8333333333333334, Média Recall@6: 0.012195121951219514


In [390]:
book_id = np.where(book_pivot.index == 'The Brethren')[0][0]
distance, suggestions = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1, -1), n_neighbors=6)

In [391]:
print(suggestions)

[[273 329 364 330 219 278]]


In [392]:
def recomendation(book_title):
    book_id = np.where(book_pivot.index == book_title)[0][0]
    distance, suggestions = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1, -1), n_neighbors=6)
    for i in range(len(suggestions)):
        books = (book_pivot.index[suggestions[i]])
        for j in books:
            print(j)

In [393]:
book_title = '1st to Die: A Novel'
recomendation(book_title)

1st to Die: A Novel
Along Came a Spider (Alex Cross Novels)
Cradle and All
Lightning
Remember Me
The Summons
